In [15]:
# With this operation, a comprehensive language model is downloaded from the 'spacy' library. There are three versions to it:
# - Large
# - Medium
# - Small
# In this case, we are using the smaller model, which is around 30 MB.

!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai==0.27.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 135.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [16]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [18]:
!pip install -U sentence-transformers rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [19]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

Read the data for Paris Hotels

In [26]:
import pandas as pd
df_miami1 = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/miami_hotels.csv')



In [33]:
df_miami1

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,The hotel was just great. Very comfy and extr...,GREAT STAFF
2507,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Everything during my stay at this hotel was am...,Great Place
2508,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Staff very professional and helpful. Rooms ve...,Great place centrally located and Great place ...
2509,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Great Hotel Great service location and walking...,Pleasant


In [34]:
# 1. Rename "name" → "hotelName"
df_miami1.rename(columns={'name': 'hotelName'}, inplace=True)

In [35]:
df_miami1

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,The hotel was just great. Very comfy and extr...,GREAT STAFF
2507,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Everything during my stay at this hotel was am...,Great Place
2508,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Staff very professional and helpful. Rooms ve...,Great place centrally located and Great place ...
2509,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,[],25.774790,-80.13631,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Great Hotel Great service location and walking...,Pleasant


In [36]:
import pandas as pd
df_miami2 = pd.read_csv('https://raw.githubusercontent.com/danielkellen6/MABA6490-Assignment2/refs/heads/main/hotelReviewsInMiami__en2019100120191005.csv')


In [37]:
df_miami2

,Unnamed: 0,review_body,review_date,hotelName,hotelUrl
0,0,Great hotel to party over the 4thj of July wee...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
1,1,Awesome place to stay at the beach. Great hote...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
2,2,There are few words to accurately depict Marth...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
3,3,Danni at the pool is awesome! Her customer ser...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
4,4,Know that this is an expensive hotel. It is a ...,"July 9, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
...,...,...,...,...,...
1795,1795,"Amazing hotel, staff outstanding and friendly....","April 29, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...
1796,1796,The non-descript hotel front belies the luxury...,"April 18, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...
1797,1797,The Setai is incredible and I will never stay ...,"April 18, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...
1798,1798,We are staying at the Setai in Miami at the mo...,"April 11, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...


In [38]:
# 2. Rename "review_body" → "extra_review"
df_miami2.rename(columns={'review_body': 'extra_review'}, inplace=True)

In [39]:
df_miami2

,Unnamed: 0,extra_review,review_date,hotelName,hotelUrl
0,0,Great hotel to party over the 4thj of July wee...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
1,1,Awesome place to stay at the beach. Great hote...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
2,2,There are few words to accurately depict Marth...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
3,3,Danni at the pool is awesome! Her customer ser...,"July 12, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
4,4,Know that this is an expensive hotel. It is a ...,"July 9, 2019",0 Eden Roc Miami Beach Resort\n2 Eden Ro...,http://www.tripadvisor.com/Hotel_Review-g34439...
...,...,...,...,...,...
1795,1795,"Amazing hotel, staff outstanding and friendly....","April 29, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...
1796,1796,The non-descript hotel front belies the luxury...,"April 18, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...
1797,1797,The Setai is incredible and I will never stay ...,"April 18, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...
1798,1798,We are staying at the Setai in Miami at the mo...,"April 11, 2018","31 The Setai, Miami Beach\nName: hotel_name...",http://www.tripadvisor.com/Hotel_Review-g34439...


# Combine Hotel Miami 1 and Hotel Miami 2 from different github sources

In [53]:
# If you only need, say, df2['value_col'] in df1:
# Perform a left join on "code" (keeps all rows from df1)

df_combined = pd.merge(df_miami1, df_miami2, on='hotelName', how='left')

# Save to CSV in your Colab workspace:
df_combined.to_csv('df_combined.csv', index=False)

In [55]:
df_combined.head()


,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,website,rankingString,rankingDenominator,numberOfReviews,review,title,Unnamed: 0,extra_review,review_date,hotelUrl
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,NaN
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,NaN
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,NaN
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,NaN
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,NaN


In [56]:
# display each shape with a label
print("df_miami1 shape:", df_miami1.shape)
print("df_miami2 shape:", df_miami2.shape)
print("df_combined shape:", df_combined.shape)


df_miami1 shape: (2511, 27)
df_miami2 shape: (1800, 5)
df_combined shape: (2511, 31)


In [57]:
# Count the number of times a specific name appears in the 'name' column, and relay that number for each different name, in order.

df_combined.hotelName.value_counts()

,count
hotelName,
Faena Miami Beach,45
1 Hotel South Beach,45
The Kent Hotel,45
The Goodtime Hotel,45
The Betsy South Beach,45
...,...
"The Julia Hotel, Miami Beach",6
Bikini Lodge,4
Beach Park Hotel,4


In [58]:
# Drop the duplicated values from each column, i.e. drop a row if it contains a duplicated value.

df_combined=df_combined.drop_duplicates()

In [97]:
# Drop unnamed:0 column

df_combined = df_combined.drop(columns=['Unnamed: 0'])



In [98]:
df_combined.shape

(2511, 31)

Create a new folder in Google Drive - called "Semantic_Search"

In [99]:
df_combined.shape

(2511, 31)

In [100]:
# Make a folder in your drive folder called "Semantic_Search".
!mkdir /content/drive/MyDrive/Semantic_Search

mkdir: cannot create directory ‘/content/drive/MyDrive/Semantic_Search’: File exists


In [101]:
# Save the dataframe to the folder you just creagted.
df_combined.to_csv('/content/drive/MyDrive/Semantic_Search/miami_combined.csv',index=False)

In [102]:
df_combined.shape

(2511, 31)

In [103]:
df_combined

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,website,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,NaN
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,NaN
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,NaN
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,NaN
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,The hotel was just great. Very comfy and extr...,GREAT STAFF,NaN,NaN,NaN,NaN
2507,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Everything during my stay at this hotel was am...,Great Place,NaN,NaN,NaN,NaN
2508,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Staff very professional and helpful. Rooms ve...,Great place centrally located and Great place ...,NaN,NaN,NaN,NaN
2509,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Great Hotel Great service location and walking...,Pleasant,NaN,NaN,NaN,NaN


In [105]:
# Create a column named 'combined', which containes the titles of the different lodges, with the descriptions associated to it.
df_combined["combined"] = (
    "title: " + df_combined.title.str.strip()+"; Content: " + df_combined.review.str.strip()
)


In [95]:
df_combined.head()

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,rankingString,rankingDenominator,numberOfReviews,review,title,Unnamed: 0,extra_review,review_date,hotelUrl,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,NaN,title: My favorite hotel in Miami--Petar is th...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,NaN,title: Exquisite Stay; Content: There are not ...
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,NaN,title: Burlesqu Show; Content: I went for the ...
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,NaN,title: Dana at Gitano is a rockstar!; Content:...
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,NaN,title: Perfect Place; Content: I would like to...


# Create Three combined title + review + extra_review but if extra_review is NaN then skip combining

In [106]:
import numpy as np

# 1) Fill title & content so they never produce NaN
titles  = df_combined['title'].fillna('').str.strip()
content = df_combined[reviews_text_col].fillna('').str.strip()
extra   = df_combined['extra_review'].fillna('').str.strip()

# 2) Build the combined column
df_combined['combined'] = (
      "title: "    + titles
    + "; Content: "+ content
    + np.where(
        df_combined['extra_review'].notna(),
        "; Extra Review: " + extra,
        ""
      )
)

# (Optional) if you’d rather have TRUE NaN when both title & content were empty:
mask_empty = df_combined['combined'].str.replace(r'\s+','',regex=True)==''
df_combined.loc[mask_empty, 'combined'] = np.nan


In [107]:
df_combined

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,website,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title: My favorite hotel in Miami--Petar is th...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title: Exquisite Stay; Content:
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title: Burlesqu Show; Content:
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title: Dana at Gitano is a rockstar!; Content:
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title: Perfect Place; Content:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,The hotel was just great. Very comfy and extr...,GREAT STAFF,NaN,NaN,NaN,title: GREAT STAFF; Content:
2507,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Everything during my stay at this hotel was am...,Great Place,NaN,NaN,NaN,title: Great Place; Content:
2508,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Staff very professional and helpful. Rooms ve...,Great place centrally located and Great place ...,NaN,NaN,NaN,title: Great place centrally located and Great...
2509,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,Great Hotel Great service location and walking...,Pleasant,NaN,NaN,NaN,title: Pleasant; Content:


In [109]:
import re

# Instead of df.copy(), use df_combined.copy()
df_combined = df_combined.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

# Translate all the "combined" column to lower case.
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))

In [110]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

GPU Found!


In [112]:
df_combined.head()

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,website,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title my favorite hotel in miamipetar is the b...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title exquisite stay content
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title burlesqu show content
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title dana at gitano is a rockstar content
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title perfect place content


In [113]:
# Take a sample of the first 10 rows.
sample = df_combined[:10]

In [114]:
sample

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,website,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title my favorite hotel in miamipetar is the b...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title exquisite stay content
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title burlesqu show content
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title dana at gitano is a rockstar content
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title perfect place content
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,NaN,NaN,NaN,title stay at faena hotel content
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,NaN,NaN,NaN,title overrated there are better luxury option...
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,NaN,NaN,NaN,title 5 star service all around content
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,My girlfriend and I had the dreamiest stay! Ev...,The perfect lover’s getaway,NaN,NaN,NaN,title the perfect lovers getaway content
9,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,My life partner and I decided to splurge and s...,Bringing back the love!,NaN,NaN,NaN,title bringing back the love content


In [115]:
# Switch to CPU
embedder =  embedder.to('cpu')

startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 1.6502480506896973


<ipython-input-115-8160b5948b46>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [116]:
# Switch once more to GPU.
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Move the embedder to the GPU
    embedder = embedder.to('cuda')
    print("GPU is available and being used.")
else:
    print("GPU not available, using CPU instead.")

# Record the start time
startTime = time.time()

# Apply the embedder to the 'combined' column of the sample DataFrame
sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

# Calculate the execution time
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

# Notice the difference in time to do this operation between GPU and CPU.

GPU is available and being used.
Execution time in seconds: 0.977757453918457


<ipython-input-116-6965480b662a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [117]:
# Switch once more to GPU.

embedder =  embedder.to('cuda')
startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

# Notice the difference in time to do this operation between GPU and CPU.


Execution time in seconds: 0.12494540214538574


<ipython-input-117-c2ada10f2885>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [118]:
sample

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title my favorite hotel in miamipetar is the b...,"[-0.0027450863, 0.0206493, -0.02324081, 0.0220..."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title exquisite stay content,"[0.05980581, 0.08602104, -0.021229682, 0.03123..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title burlesqu show content,"[0.021778863, 0.006451166, -0.027481794, 0.067..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title dana at gitano is a rockstar content,"[0.07053833, 0.037569977, -0.006891556, 0.0709..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title perfect place content,"[0.07631112, 0.048725203, -0.03492778, 0.01106..."
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,NaN,NaN,NaN,title stay at faena hotel content,"[0.040106352, -0.017388249, -0.026944455, 0.06..."
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,NaN,NaN,NaN,title overrated there are better luxury option...,"[-0.019298369, 0.038705476, -0.0066358247, 0.0..."
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,NaN,NaN,NaN,title 5 star service all around content,"[0.055280097, -0.002862984, -0.0018693515, 0.0..."
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,My girlfriend and I had the dreamiest stay! Ev...,The perfect lover’s getaway,NaN,NaN,NaN,title the perfect lovers getaway content,"[0.07618872, 0.044704117, -0.016958324, 0.0472..."
9,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,My life partner and I decided to splurge and s...,Bringing back the love!,NaN,NaN,NaN,title bringing back the love content,"[0.08437964, 0.0715512, -0.015334807, -0.00406..."


In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [119]:
# Transform your dataframe to a pickle file, which is a byte stream file used to save a dataframe's state across sections.
sample.to_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl')    #to save the dataframe, df to 123.pkl

In [120]:
# Load the pickle file.
df_with_embedding = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl') #to load 123.pkl back to the dataframe df

In [121]:
df_with_embedding.head()

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title my favorite hotel in miamipetar is the b...,"[-0.0027450863, 0.0206493, -0.02324081, 0.0220..."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title exquisite stay content,"[0.05980581, 0.08602104, -0.021229682, 0.03123..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title burlesqu show content,"[0.021778863, 0.006451166, -0.027481794, 0.067..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title dana at gitano is a rockstar content,"[0.07053833, 0.037569977, -0.006891556, 0.0709..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title perfect place content,"[0.07631112, 0.048725203, -0.03492778, 0.01106..."


In [122]:
query = 'Good price'

# Embed the previous query.
query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [123]:
# First element of the 'combined' column.
df_with_embedding.combined[0]

'title my favorite hotel in miamipetar is the best content '

In [124]:
# Create a list of embeddings, from the contents of the "combined" column.
corpus_embeddings = embedder.encode(df_with_embedding.combined,show_progress_bar=True)

# Create a column called 'similarity', displaying the cosine similarity between your embedded query and each embedded content from the 'corpus_embeddings' variable.
df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [125]:
df_with_embedding

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined,embedding,similarity
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title my favorite hotel in miamipetar is the b...,"[-0.0027450863, 0.0206493, -0.02324081, 0.0220...",0.032245
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title exquisite stay content,"[0.05980581, 0.08602104, -0.021229682, 0.03123...",0.045945
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title burlesqu show content,"[0.021778863, 0.006451166, -0.027481794, 0.067...",0.029411
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title dana at gitano is a rockstar content,"[0.07053833, 0.037569977, -0.006891556, 0.0709...",0.023111
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title perfect place content,"[0.07631112, 0.048725203, -0.03492778, 0.01106...",0.042166
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,NaN,NaN,NaN,title stay at faena hotel content,"[0.040106352, -0.017388249, -0.026944455, 0.06...",0.015119
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,NaN,NaN,NaN,title overrated there are better luxury option...,"[-0.019298369, 0.038705476, -0.0066358247, 0.0...",0.035165
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,NaN,NaN,NaN,title 5 star service all around content,"[0.055280097, -0.002862984, -0.0018693515, 0.0...",0.050322
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,My girlfriend and I had the dreamiest stay! Ev...,The perfect lover’s getaway,NaN,NaN,NaN,title the perfect lovers getaway content,"[0.07618872, 0.044704117, -0.016958324, 0.0472...",0.024152
9,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,235,2123,My life partner and I decided to splurge and s...,Bringing back the love!,NaN,NaN,NaN,title bringing back the love content,"[0.08437964, 0.0715512, -0.015334807, -0.00406...",0.008617


In [ ]:
# cosine_similarity(corpus_embeddings, query_embedding)

#Entire Data

In [126]:
startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 34.21678185462952


In [127]:
# Turn the dataframe you have just created to pickle file, for later use.
df_combined.to_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl')

##Embedding upload - new starting point


In [128]:
!pip install openai==0.27.7
!pip install gradio
!pip install -U sentence-transformers

In [129]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [131]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

GPU Found!


In [132]:
# Read the pickle file you entered earlier.
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [133]:
df.shape

(2511, 32)

In [134]:
df.head()

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,rankingString,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,NaN,NaN,NaN,title my favorite hotel in miamipetar is the b...,"[-0.0027450863, 0.0206493, -0.02324081, 0.0220..."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,NaN,NaN,NaN,title exquisite stay content,"[0.05980581, 0.08602104, -0.021229682, 0.03123..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,NaN,NaN,NaN,title burlesqu show content,"[0.021778863, 0.006451166, -0.027481794, 0.067..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,NaN,NaN,NaN,title dana at gitano is a rockstar content,"[0.07053833, 0.037569977, -0.006891556, 0.0709..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,NaN,NaN,NaN,title perfect place content,"[0.07631112, 0.048725203, -0.03492778, 0.01106..."


In [135]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):

    # Embed your search query.
    query_embedding = embedder.encode(query,show_progress_bar=True)

    # As before, create a 'similarity' column, which shows the cosine similarity between the your query and the embedded combined contents.
    # REMEMBER!! YOU ARE USING THE SAME MODEL TO EMBED BOTH THE COMBINED CONTENTS AND YOUR QUERY.
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1))) #similarity against each doc

    # Now, sort the values bty similarity, and choose the most similar doc.
    results = (
        df.sort_values("similarity", ascending=False) # re-rank
        .head(n))

    return results

In [136]:
query = 'hotel with the nice sea view'

In [137]:
results = search_reviews(df,query,15)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [138]:
results

,id,type,hotelName,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,rankingDenominator,numberOfReviews,review,title,extra_review,review_date,hotelUrl,combined,embedding,similarity
1084,15346944,HOTEL,"Life House, South of Fifth",https://media-cdn.tripadvisor.com/media/photo-...,[],158,$$$,$194 - $397,hotel,4.0,...,235,40,Definitely stay here if you are looking for a ...,Beautiful boutique hotel - located near all th...,NaN,NaN,NaN,title beautiful boutique hotel located near a...,"[0.02904009, 0.014884778, -0.01310008, 0.02583...",[0.7037934]
631,2316201,HOTEL,The Shepley Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],42,$$$,$193 - $319,hotel,4.5,...,235,980,Such a quaint hotel with great staff and servi...,Best hotel near the beach 🏝,NaN,NaN,NaN,title best hotel near the beach content,"[0.030845268, -0.011401449, -0.013642973, 0.04...",[0.6954652]
1885,613545,HOTEL,Sonder The Abbey,https://media-cdn.tripadvisor.com/media/photo-...,[],55,$$$,$141 - $261,hotel,4.5,...,235,119,I love the location. Walking distance to the c...,Quiet quaint hotel close to the beach- perfect...,NaN,NaN,NaN,title quiet quaint hotel close to the beach pe...,"[0.02225834, 0.016413657, 0.0034965004, 0.0270...",[0.6897511]
1453,21318377,HOTEL,Moxy Miami South Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],53,$$$$,$227 - $398,hotel,4.5,...,235,367,We planned a short get away and wanted to chec...,Nice hotel close to beach,NaN,NaN,NaN,title nice hotel close to beach content,"[0.020449957, 0.007054828, -0.016736375, 0.031...",[0.68471754]
407,2008516,HOTEL,"Marriott Vacation Club Pulse, South Beach",https://media-cdn.tripadvisor.com/media/photo-...,[],49,$$$,$190 - $329,hotel,4.5,...,235,1864,The Marriott Vacation Club Pulse at South Beac...,Cozy Art Deco Boutique Hotel On Ocean Drive Th...,NaN,NaN,NaN,title cozy art deco boutique hotel on ocean dr...,"[0.01071928, 0.05890315, -0.0069783865, 0.0298...",[0.6607512]
2285,115184,HOTEL,The Marlin Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],36,$$$,$230 - $363,hotel,4.5,...,235,929,A gorgeous boutique hotel. Perfect location to...,Excellent place to stay in south beach! Cute b...,NaN,NaN,NaN,title excellent place to stay in south beach c...,"[0.03254927, -0.045224853, 0.009578844, 0.0134...",[0.6446793]
1168,7940209,HOTEL,The Gates Hotel South Beach - a DoubleTree by ...,https://media-cdn.tripadvisor.com/media/photo-...,[],20,$$$,$189 - $349,hotel,4.0,...,235,3217,"The Gates is a really nice hotel, they have bi...",Wonderful Hotel Near The Beach,NaN,NaN,NaN,title wonderful hotel near the beach content,"[0.031114858, 0.010272351, -0.010867056, 0.044...",[0.64300007]
2003,3937200,HOTEL,Pestana Miami South Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],21,$$$,$189 - $321,hotel,4.5,...,235,3624,5… ideally located and the team is great espec...,"The most friendly, ideally located hotel in So...",NaN,NaN,NaN,title the most friendly ideally located hotel ...,"[0.04968395, -0.024398115, 0.0026681798, 0.010...",[0.64277285]
2072,1787787,HOTEL,Hotel Breakwater South Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],15,$$$,$164 - $276,hotel,4.0,...,235,3136,Let me tell y’all.. I never leave reviews but ...,BEST and NICEST hotel you’ll find on Ocean Dri...,NaN,NaN,NaN,title best and nicest hotel youll find on ocea...,"[0.029556321, 0.056050286, -0.004642946, 0.048...",[0.6412732]
836,5600182,HOTEL,Berkeley Shore Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],40,$$$,$158 - $294,hotel,4.5,...,235,828,We stayed at the Berkeley Shore Hotel during S...,Great location for a boutique hotel in South B...,NaN,NaN,NaN,title great location for a boutique hotel in s...,"[0.039038606, -0.02277017, 0.0021523393, 0.014...",[0.6330831]


Take all the reviews which are closest to the query, and groupby the hotel name

In [143]:
def search(query):
  # Define a number of results to return, in this case, return only the first 15 results ranked by similarity.
  n = 15

  # Embed the query.
  query_embedding = embedder.encode(query)

  # Generate the similarity column, based on your query.
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  # Calculate the top 'n' most similar results by similarity.
  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  # Display them in a very concise and ordered manner.
  hlist = []
  for r in results.index:
      # Change 'results.name[r]' to 'results.hotelName[r]' or 'results.loc[r, "hotelName"]' to access the hotel name
      if results.hotelName[r] not in hlist:
          smalldf = results.loc[results.hotelName == results.hotelName[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.hotelName[r],  # Access hotel name using 'hotelName' column
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.review[s] for s in smalldf.index] # Changed 'text' to 'review'
          })
          hlist.append(results.hotelName[r])  # Access hotel name using 'hotelName' column
  return resultlist

In [144]:
search('hotel with the great beach view')

[{'name': 'Life House, South of Fifth',
  'score': np.float32(0.78892064),
  'rating': 4.0,
  'relevant_reviews': ['Definitely stay here if you are looking for a place stay at south beach. The hotel is so beautiful and so clean- the beach is less than 5 min walk away and the best places to eat are all a walking distance away. The staff are really nice and will gives you amazing recommendations on places to eat. There are live music some nights and the food at the hotel is amazing as well!']},
 {'name': 'The Shepley Hotel',
  'score': np.float32(0.781044),
  'rating': 4.5,
  'relevant_reviews': ['Such a quaint hotel with great staff and service. Carolina made sure our room was perfect. Jorge and Gabi, Ariel and Laralo made sure our stay was impeccable. Tony gave us great restaurant recommendations. We are sure to return to The Shepley!']},
 {'name': 'Sonder The Abbey',
  'score': np.float32(0.75711316),
  'rating': 4.5,
  'relevant_reviews': ['I love the location. Walking distance to th

In [176]:
from google.colab import userdata
import openai
openai.api_key = userdata.get('open_ai_key')


In [177]:
def generate_answer(query):
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context
    focusing on which is the top ten hotel based on the query. You should respond as you are a travel agent and are conversing with the
    user in a nice beach view. Always address the user as Travis. Make sure you infrom the user why this is a good answer. The answer should be a paragph
    remove the special characters and (/n ) , make the output clean and concise.
    Answer only as a poet.


    ###########
    query:
    "{query}"

    ########

    context:"
    "{search(query)}"
    #####

    Return in Markdown format with each hotel highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.2, #higher temperature means more creative or more hallucination
        messages = messages


    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message['content'].strip()

    return generated_text

# # Example usage
# query = "What are the best amenities offered by Hotel XYZ?"
# response = generate_hotel_response(query)
# print(response)

    #return response.choices[0].message.content.strip()

In [178]:
txt=generate_answer('hotel with nice beach view')

In [ ]:
# prompt

In [179]:
import markdown
from IPython.display import display, HTML

def render_markdown(md_text):
    # Convert Markdown to HTML
    html = markdown.markdown(md_text)
    # Display the HTML
    display(HTML(html))

In [180]:
render_markdown(txt)

#Building the API

In [181]:

import gradio as gr

# def search(query):
#   n = 15
#   query_embedding = embedder.encode(query)
#   df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

#   results = (
#       df.sort_values("similarity", ascending=False)
#       .head(n))

#   resultlist = []

#   hlist = []
#   for r in results.index:
#       if results.name[r] not in hlist:
#           smalldf = results.loc[results.name == results.name[r]]
#           smallarr = smalldf.similarity[r].max()
#           sm =smalldf.rating[r].mean()

#           if smalldf.shape[1] > 3:
#             smalldf = smalldf[:3]

#           resultlist.append(
#           {
#             "name":results.name[r],
#             "description":results.description[r],
#             "relevance score": smallarr.tolist(),
#             "rating": sm.tolist(),
#             "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
#           })
#           hlist.append(results.name[r])
#   return resultlist

def greet(query):
    bm25 = generate_answer(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)
# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c62c5f75316679560a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [182]:
!pip install --upgrade gradio

# Explore Gradio Simple Demo Chatbot

In [183]:
import gradio as gr

# ... your greet function and other code ...

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)  # Relaunch the Gradio app
# COPY THE NEW URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3fd426cfe0ea57c8cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [184]:
import gradio as gr

def greet(name):
    return f"Hello, {name}!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# By setting share=True, Gradio will print a public URL when you run this cell.
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://443fbd43b3b6b154fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [185]:
# Import the Client class from the gradio_client module.
from gradio_client import Client

# Create an instance of the Client class. The URL provided should point to a live Gradio app.

# PASTE THE URL FROM ABOVE HERE
client = Client("https://c62c5f75316679560a.gradio.live")

# Use the 'predict' method of the Client instance to send a request to the Gradio app.
# The string "Hotel near the Eiffel Tower!" is passed to the 'query' textbox component of the app.
# 'api_name' specifies the endpoint ('/predict') that the Gradio interface exposes for processing this input.
result = client.predict(
				"Hotel with the best beach view!",	# str in 'query' Textbox component
				api_name="/predict"
)

# Print the result returned from the Gradio app. This output depends on how the Gradio app processes the input.
print(result)



Loaded as API: https://c62c5f75316679560a.gradio.live/ ✔
Travis, as you gaze upon the serene waves and feel the gentle breeze, let me share with you the top ten hotels that boast the best beach views, perfect for your getaway. 

1. **The Shepley Hotel** - A quaint gem with impeccable service, just a stroll from the beach.
2. **Pestana Miami South Beach** - Ideally located, where the team makes you feel at home, with a pool experience to savor.
3. **Life House, South of Fifth** - Beautiful and clean, a mere five-minute walk to the beach, with delightful dining nearby.
4. **Sonder The Abbey** - A quiet retreat close to the beach, offering comfort and charm at a reasonable price.
5. **The Marlin Hotel** - Stunning rooms and exceptional customer service, just two minutes from the beach.
6. **Berkeley Shore Hotel** - Great location with a rooftop pool and delicious dining options, perfect for exploring South Beach.
7. **Moxy Miami South Beach** - A short walk to the beach, with bike rentals

In [186]:
result

'Travis, as you gaze upon the serene waves and feel the gentle breeze, let me share with you the top ten hotels that boast the best beach views, perfect for your getaway. \n\n1. **The Shepley Hotel** - A quaint gem with impeccable service, just a stroll from the beach.\n2. **Pestana Miami South Beach** - Ideally located, where the team makes you feel at home, with a pool experience to savor.\n3. **Life House, South of Fifth** - Beautiful and clean, a mere five-minute walk to the beach, with delightful dining nearby.\n4. **Sonder The Abbey** - A quiet retreat close to the beach, offering comfort and charm at a reasonable price.\n5. **The Marlin Hotel** - Stunning rooms and exceptional customer service, just two minutes from the beach.\n6. **Berkeley Shore Hotel** - Great location with a rooftop pool and delicious dining options, perfect for exploring South Beach.\n7. **Moxy Miami South Beach** - A short walk to the beach, with bike rentals and a vibrant atmosphere.\n8. **Marriott Vacati